In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS']='--driver-memory 8G --executor-memory 8G pyspark-shell'
# spark-shell --driver-memory 10G --executor-memory 15G --executor-cores 8
# --deploy-mode client --driver-memory 12G
# --files https://ifood-data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz
# spark=SparkSession.builder.config('spark.executor.memory','6g').config('spark.driver.memory','6g').getOrCreate()
# --master yarn 


In [1]:
## start a spark session 
import findspark 
findspark.init() 
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from os.path import abspath

warehouse_location = abspath('spark-warehouse')

spark=SparkSession.builder.appName("ETL pipeline")\
.config("spark.sql.warehouse.dir", warehouse_location)\
.config("spark.io.encryption.enabled",True)\
.enableHiveSupport().getOrCreate()
sc=spark.sparkContext
spark


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/07 23:23:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
bucket='https://ifood-data-architect-test-source.s3-sa-east-1.amazonaws.com/'
file=['consumer.csv.gz','restaurant.csv.gz','order.json.gz'][1]

# sc.addFile(bucket+file)
file_path=f'file://{SparkFiles.get(file)}'
df=spark.read.option('header',True).option("inferSchema",True).csv(file_path)
# store_path=f'./data/{file.split(".")[0]}'
# df.write.parquet(store_path)

In [ ]:
df=spark.read.parquet('data/restaurant')
# df.printSchema()

In [ ]:
df.createOrReplaceTempView('temp_restaurant')

In [ ]:
spark.sql("drop table if exists restaurant");
spark.sql("create table restaurant as select * from temp_restaurant");


In [ ]:
spark.sql('show databases').show()

In [ ]:
# spark.sql('select * from restaurant limit 10').show()
spark.sql('describe formatted restaurant').show()

In [ ]:
spark.sql('select * from temp_restaurant').show()